In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
import dash_table


In [2]:
import pandas as pd  
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


In [3]:
##########################################################
#Get the financial data for IS, CF, BS
#########################################################

def income_statement_fx(x):
    url_stem = 'https://finance.yahoo.com/quote/'
    url_full = url_stem + ticker+'/financials'
    response = requests.get(url_full)
    soup = BeautifulSoup(response.content, 'lxml')
    income_statement_data = soup.find_all('div', {'class':'D(tbr)'})
    return income_statement_data

def balance_sheet_fx(x):
    url_stem = 'https://finance.yahoo.com/quote/'
    url_full = url_stem + ticker+'/balance-sheet'
    response = requests.get(url_full)
    soup = BeautifulSoup(response.content, 'lxml')
    balance_sheet_data = soup.find_all('div', {'class':'D(tbr)'})
    return balance_sheet_data

def cash_flow_fx(x):
    url_stem = 'https://finance.yahoo.com/quote/'
    url_full = url_stem + ticker+'/cash-flow'
    response = requests.get(url_full)
    soup = BeautifulSoup(response.content, 'lxml')
    cash_flow_data = soup.find_all('div', {'class':'D(tbr)'})
    return cash_flow_data

In [4]:
##########################################################
#Get the columns separate from the rows, will rename later
#Income Statement Extraction
#########################################################

def income_statement(x):
    columns = []
    for x in income_statement_data[0]:
        temp = x.text
        columns.append(temp)

    rows = []
    for x in income_statement_data:
        temp = x.find_all('div', class_= 'D(tbc)')
        rows.append(temp)
    rows = rows[1:] #Get rid of the first row since it's just None
    r = []
    for x in rows:
        s = []
        for i in x:
            temp = i.text
            s.append(temp)
        r.append(s)
    is_df = pd.DataFrame(r)
    is_df.columns = columns
    is_df = is_df.set_index('Breakdown')
    return is_df

################################################################################################################
#The stuff python recognizes as numbers is pretty dumb, so you have to get rid of commas, hyphens, blank spaces
#############################################################################################################
def clean_is(x):
    columns = list(is_df.columns)
    for x in columns:
        is_df[x] = is_df[x].str.replace(',','')
        is_df[x] = is_df[x].str.replace('-','')
        is_df[x] = is_df[x].replace(r'^\s*$', np.nan, regex=True)
        is_df[x] = is_df[x].fillna(0)
        is_df[x] = is_df[x].astype(float)
    return is_df[is_df.columns[::-1]]*1000 #Reverse order of the columns 


##########################################################
#Balance Sheet extraction
#########################################################

def balance_sheet(x):
    columns = []
    for x in balance_sheet_data[0]:
        temp = x.text
        columns.append(temp)

    rows = []
    for x in balance_sheet_data:
        temp = x.find_all('div', class_= 'D(tbc)')
        rows.append(temp)
    rows = rows[1:] #Get rid of the first row since it's just None
    r = []
    for x in rows:
        s = []
        for i in x:
            temp = i.text
            s.append(temp)
        r.append(s)
    bs_df = pd.DataFrame(r)
    bs_df.columns = columns
    bs_df = bs_df.set_index('Breakdown')
    return bs_df

################################################################################################################
#The stuff python recognizes as numbers bs pretty dumb, so you have to get rid of commas, hyphens, blank spaces
#############################################################################################################
def clean_bs(x):
    columns = list(bs_df.columns)
    for x in columns:
        bs_df[x] = bs_df[x].str.replace(',','')
        bs_df[x] = bs_df[x].str.replace('-','')
        bs_df[x] = bs_df[x].replace(r'^\s*$', np.nan, regex=True)
        bs_df[x] = bs_df[x].fillna(0)
        bs_df[x] = bs_df[x].astype(float)
    return bs_df[bs_df.columns[::-1]]*1000


##########################################################
#CF extraction
#########################################################

def cash_flow(x):
    columns = []
    for x in cash_flow_data[0]:
        temp = x.text
        columns.append(temp)

    rows = []
    for x in cash_flow_data:
        temp = x.find_all('div', class_= 'D(tbc)')
        rows.append(temp)
    rows = rows[1:] #Get rid of the first row since it's just None
    r = []
    for x in rows:
        s = []
        for i in x:
            temp = i.text
            s.append(temp)
        r.append(s)
    cf_df = pd.DataFrame(r)
    cf_df.columns = columns
    cf_df = cf_df.set_index('Breakdown')
    return cf_df

################################################################################################################
#The stuff python recognizes as numbers cf pretty dumb, so you have to get rid of commas, hyphens, blank spaces
#############################################################################################################
def clean_cf(x):
    columns = list(cf_df.columns)
    for x in columns:
        cf_df[x] = cf_df[x].str.replace(',','')
        cf_df[x] = cf_df[x].str.replace('-','')
        cf_df[x] = cf_df[x].replace(r'^\s*$', np.nan, regex=True)
        cf_df[x] = cf_df[x].fillna(0)
        cf_df[x] = cf_df[x].astype(float)
    return cf_df[cf_df.columns[::-1]]*1000


In [5]:
def analyst_reports(x):
    #Analyst Estimates
    url = 'https://finance.yahoo.com/quote/{}/analysis'.format(ticker)
    analyst_consensus_data = requests.get(url)
    soup = BeautifulSoup(analyst_consensus_data.text, 'lxml')
    #First get all the tables and put them into a list 
    tables = soup.select('table')
    #########################
    #Earnings Table
    #########################

    earnings_table = tables[0]

    #Columns
    earnings_headers = earnings_table.find_all('th')
    earnings_columns = []
    for x in earnings_headers:
        earnings_columns.append(x.text)

    #Rows
    earnings_rows = []
    for row in earnings_table.find_all('tr'):
        earnings_text = [x.text for x in row.find_all('td')]
        earnings_rows.append(earnings_text)

    #Now put them into a dataframe
    earnings_df = pd.DataFrame(earnings_rows[1:]) #first row is blank, so get rid of it
    earnings_df.columns = earnings_columns

    #########################
    #Revenue Table
    #########################

    revenue_table = tables[1]

    #Columns
    revenue_headers = revenue_table.find_all('th')
    revenue_columns = []
    for x in revenue_headers:
        revenue_columns.append(x.text)

    #Rows
    revenue_rows = []
    for row in revenue_table.find_all('tr'):
        revenue_text = [x.text for x in row.find_all('td')]
        revenue_rows.append(revenue_text)

    #Now put them into a dataframe
    revenue_df = pd.DataFrame(revenue_rows[1:]) #first row is blank, so get rid of it
    revenue_df.columns = revenue_columns

    #########################
    #Earnings History
    #########################
    earning_history_table = tables[2]

    #Columns
    earning_history_headers = earning_history_table.find_all('th')
    earning_history_columns = []
    for x in earning_history_headers:
        earning_history_columns.append(x.text)

    #Rows
    earning_history_rows = []
    for row in earning_history_table.find_all('tr'):
        earning_history_text = [x.text for x in row.find_all('td')]
        earning_history_rows.append(earning_history_text)

    #Now put them into a dataframe
    earning_history_df = pd.DataFrame(earning_history_rows[1:]) #first row is blank, so get rid of it
    earning_history_df.columns = earning_history_columns

    #########################
    #EPS Trend 
    #########################

    eps_trend_table = tables[3]

    #Columns
    eps_trend_headers = eps_trend_table.find_all('th')
    eps_trend_columns = []
    for x in eps_trend_headers:
        eps_trend_columns.append(x.text)

    #Rows
    eps_trend_rows = []
    for row in eps_trend_table.find_all('tr'):
        eps_trend_text = [x.text for x in row.find_all('td')]
        eps_trend_rows.append(eps_trend_text)

    #Now put them into a dataframe
    eps_trend_df = pd.DataFrame(eps_trend_rows[1:]) #first row is blank, so get rid of it
    eps_trend_df.columns = eps_trend_columns

    #########################
    #EPS Revisions 
    #########################
    eps_revisions_table = tables[4]

    #Columns
    eps_revisions_headers = eps_revisions_table.find_all('th')
    eps_revisions_columns = []
    for x in eps_revisions_headers:
        eps_revisions_columns.append(x.text)

    #Rows
    eps_revisions_rows = []
    for row in eps_revisions_table.find_all('tr'):
        eps_revisions_text = [x.text for x in row.find_all('td')]
        eps_revisions_rows.append(eps_revisions_text)

    #Now put them into a dataframe
    eps_revisions_df = pd.DataFrame(eps_revisions_rows[1:]) #first row is blank, so get rid of it
    eps_revisions_df.columns = eps_revisions_columns

    #########################
    #Growth Estimates 
    #########################

    growth_estimates_table = tables[5]

    #Columns
    growth_estimates_headers = growth_estimates_table.find_all('th')
    growth_estimates_columns = []
    for x in growth_estimates_headers:
        growth_estimates_columns.append(x.text)

    #Rows
    growth_estimates_rows = []
    for row in growth_estimates_table.find_all('tr'):
        growth_estimates_text = [x.text for x in row.find_all('td')]
        growth_estimates_rows.append(growth_estimates_text)

    #Now put them into a dataframe
    growth_estimates_df = pd.DataFrame(growth_estimates_rows[1:]) #first row is blank, so get rid of it
    growth_estimates_df.columns = growth_estimates_columns
    
    return earnings_df, revenue_df, earning_history_df, eps_trend_df,eps_revisions_df, growth_estimates_df

In [6]:
#Use if running locally, and not in app
if __name__ == '__main__':
    ticker = input('Please input the stock of interest: ')
    income_statement_data = income_statement_fx(ticker)
    is_df = income_statement(income_statement_data)
    is_df = clean_is(is_df)
    balance_sheet_data = balance_sheet_fx(ticker)
    bs_df = balance_sheet(balance_sheet_data)
    bs_df = clean_bs(bs_df)
    cash_flow_data= cash_flow_fx(ticker)
    cf_df = cash_flow(cash_flow_data)
    cf_df = clean_cf(cf_df)
    earnings_df, revenue_df, earning_history_df, eps_trend_df,eps_revisions_df, growth_estimates_df = analyst_reports(ticker)

Please input the stock of interest: mdgl


In [7]:
#########################################
#FCF and financial analysis testing
#######################################
def fcf_calc(x):
    fcf_data = x.reset_index()
    #if  fcf_data[(fcf_data['Breakdown']=='Cost of Revenue')].iloc[0,1:].notnull().any(): #this didn't work
    try:   
        fcf_data = is_df.reset_index()
        fcf_df = fcf_data.loc[(fcf_data['Breakdown']== 'Total Revenue')|(fcf_data['Breakdown']=='Cost of Revenue')]
        fcf_df= fcf_df.set_index('Breakdown')
        fcf_df =fcf_df.T #easier to work with columns
        fcf_df['Cost of Revenue'] = - fcf_df['Cost of Revenue']
        #Calculate gross profit
        fcf_df['Gross Profit'] = fcf_df['Total Revenue'] + fcf_df['Cost of Revenue']
        #Calculate EBIT (using EBIT = Revenue -COGS -Operating Expenses)
        oper_exp = fcf_data.loc[(fcf_data['Breakdown']== 'Operating Expense')].set_index('Breakdown')
        fcf_df['Operating Expense'] = oper_exp.T
        fcf_df['Operating Expense'] =  -fcf_df['Operating Expense']
        fcf_df['Operating Income'] = fcf_df['Gross Profit']+ fcf_df['Operating Expense']
        other_income_exp = fcf_data.loc[(fcf_data['Breakdown']== 'Other Income Expense')].set_index('Breakdown')
        fcf_df['Other Income Expense'] = other_income_exp.T
        fcf_df['EBIT'] = fcf_df['Operating Income']+fcf_df['Other Income Expense']
    except: #pre revenue company
        fcf_data = is_df.reset_index()
        fcf_df = fcf_data.loc[(fcf_data['Breakdown']== 'Total Revenue')]
        fcf_df= fcf_df.set_index('Breakdown')
        fcf_df =fcf_df.T #easier to work with columns
        #Calculate gross profit
        fcf_df['Gross Profit'] = fcf_df['Total Revenue'] 
        #Calculate EBIT (using EBIT = Revenue -COGS -Operating Expenses)
        oper_exp = fcf_data.loc[(fcf_data['Breakdown']== 'Operating Expense')].set_index('Breakdown')
        fcf_df['Operating Expense'] = oper_exp.T
        fcf_df['Operating Expense'] =  -fcf_df['Operating Expense']
        fcf_df['Operating Income'] = fcf_df['Gross Profit']+ fcf_df['Operating Expense']
        other_income_exp = fcf_data.loc[(fcf_data['Breakdown']== 'Other Income Expense')].set_index('Breakdown')
        fcf_df['Other Income Expense'] = other_income_exp.T
        fcf_df['EBIT'] = fcf_df['Operating Income']+fcf_df['Other Income Expense']

    return fcf_df.T

In [8]:
##############################################################################################
#Function to a) convert dataframe's dates to year integers, and add in forward looking columns
##############################################################################################

def int_years(x):
    last_year = pd.DataFrame((pd.DatetimeIndex(x.columns.values[0:1]).year))
    last_year= int(last_year.iloc[0,0])
    temp = x.rename(columns = {x.columns.values[0]:last_year})
    start = len(x.T)
    fwd = len(x.T)+15 #Define the number of years you will be looking forward
    temp_2 = temp.T.reset_index()
    years = pd.DataFrame({'Breakdown':range(last_year, last_year+fwd)})
    temp3 = pd.concat([temp_2, years], axis = 1)
    temp3 = temp3.set_index('Breakdown').drop(columns = ['index']).T
    return temp3

In [9]:
#############
#FCF Analysis
#############
def fcf_fwd(x):
    temp= fcf_df2 .T

    #Change in Revenue 

    temp['rev growth'] = temp['Total Revenue']/temp['Total Revenue'].shift(1)
    temp['rev growth'] = temp['rev growth'].fillna(method='ffill') #Fill foward assuming last value

    try:
        #COGS as % of Revenue
        temp['COGs%'] = -temp['Cost of Revenue']/temp['Total Revenue'] 
        temp['COGs%'] = temp['COGs%'].fillna(method='ffill') #Fill foward assuming last value
    except KeyError: #unless it's pre revenue
        pass


    #Operating expense as % of Gross Profit
    temp['OE%'] = -temp['Operating Expense']/temp['Gross Profit'] 
    temp['OE%'] = temp['OE%'].fillna(method='ffill')#Fill foward assuming last value

    #Just fill in the last 'Other Income Expense, or fill as 0's if you're confident it doesn't impact the valuation
    temp['Other Income Expense'] = temp['Other Income Expense'].fillna(method = 'ffill')
    #Now calculate the foward looking revenue and cost of revenue using previously calculated rev growth and COGs respectively
    #iterate over the rows (revenue next year = revenue previous year * revenue growth) 
    #I just copied the most recent revenue growth forward
    for index, row in temp.iterrows():
        temp['Total Revenue'].iloc[1:] = temp['Total Revenue'].shift(1)*temp['rev growth'] 
        try:
            temp['Cost of Revenue'].iloc[1:] = -temp['Total Revenue']*temp['COGs%']
        except KeyError:
            pass
    
    #Just recalculate the gross profit, it doesn't change for the values that exist, and calcualtes for forward looking ones
    try:
        temp['Gross Profit'] = temp['Total Revenue'] + temp['Cost of Revenue']
    except KeyError:
        temp['Gross Profit'] = temp['Total Revenue']
    #Calcualte Operating Expense by multiplying Gross Profit x Operating Expense % previously calculated
    for index, row in temp.iterrows():
        temp['Operating Expense'].iloc[1:] = -temp['Gross Profit']*temp['OE%'] 
    #Recalculate Operating Income, EBIT 
    temp['Operating Income'] = temp['Gross Profit'] + temp['Operating Expense']
    temp['EBIT'] = temp['Operating Income']
    temp = temp.T.reset_index()
    temp= temp.rename(columns = {'index':'Breakdown'})
    return temp

In [10]:
def dcf_calc(x):
    dcf_data = x.reset_index().rename(columns = {'index':'Breakdown'})
    dcf_data.loc[(dcf_data['Breakdown'] =='Tax Provision')]
    dcf_df = fcf_df_clean.reset_index()
    dcf_df = dcf_df.loc[dcf_df['Breakdown'] == 'EBIT']
    dcf_df = dcf_df.drop(columns = 'index')
    dcf_df = dcf_df.set_index('Breakdown')
    dcf_df = dcf_df.append(dcf_data.loc[(dcf_data['Breakdown'] =='Tax Provision')].set_index('Breakdown'))
    #Calculate the tax rate by dividing the tax paid/operating income from the income statement
    dcf_df = dcf_df.T
    dcf_df['tax rate'] = dcf_df['Tax Provision']/dcf_df['EBIT']
    dcf_df['tax rate'] = dcf_df['tax rate'].fillna(method = 'ffill')
    dcf_df['Tax Provision'] = -dcf_df['EBIT']* dcf_df['tax rate']

    #Yahoofinance takes a simplistic way of determining working capital: current assets - currant liabilities
    #I'll improve it when I figure out how to expand hidden items that bs4 won't load
    #THANK GOD THEY ACTUALLY MAKE THE NUMBERS EITHER POSITIVE OR NEGATIVE WHEN CALCULATED CORRECTLY
    wc_data = bs_df.reset_index()
    wc_df = wc_data.loc[wc_data['Breakdown'] == 'Working Capital']
    wc_df = wc_df.set_index('Breakdown')
    wc_df = int_years(wc_df)
    dcf_df = dcf_df.T.append(wc_df)
    
    #################################################################
    #Placeholder for working capital, need to change this
    #WC is the change year to year in inventory/payables/receivables
    ################################################################


    dcf_df = dcf_df.T
    dcf_df['wc%ebit'] = dcf_df['Working Capital']/dcf_df['EBIT']
    dcf_df['wc%ebit'] = dcf_df['wc%ebit'].fillna(method = 'ffill')
    #dcf_df['Working Capital'] = dcf_df['EBIT']*dcf_df['wc%ebit']
    dcf_df['Working Capital'] = dcf_df['Working Capital'].fillna(method = 'ffill')
    dcf_df['change_wc'] = dcf_df['Working Capital']-dcf_df['Working Capital'].shift(1)
    dcf_df = dcf_df.T
    
    ###############################################################
    #Include capex, maybe later we will want to calculate ourselves by going 
    #to financial statements and getting from PPE, other purchases
    ###############################################################
    capex_data = cf_df.reset_index()
    capex_df = capex_data.loc[capex_data['Breakdown'] == 'Capital Expenditure']
    capex_df = capex_df.set_index('Breakdown')
    #Convert years to integers to match the columns of other dataframes
    capex_df = int_years(capex_df)
    #calculate Capex as % of revenue for looking forward
    capex_df = capex_df.T
    temp = fcf_df_clean.set_index('Breakdown')
    temp = temp.T
    capex_df['capex%rev'] = capex_df['Capital Expenditure']/temp['Total Revenue']
    capex_df['capex%rev'] = capex_df['capex%rev'].fillna(method = 'ffill')
    capex_df['Capital Expenditure'] = -temp['Total Revenue']*capex_df['capex%rev']
    capex_df = capex_df.T
    dcf_df = dcf_df.append(capex_df)
    
    ############################
    #Finally calculate the FCF
    ############################

    dcf_df = dcf_df.T
    dcf_df['FCF'] = dcf_df['EBIT'] +dcf_df['Tax Provision'] +dcf_df['change_wc'].notnull() + dcf_df['Capital Expenditure']
    dcf_df = dcf_df.T
    
    return dcf_df

In [11]:
def discount_period(x, discount_rate):

    #Calculate the year current year
    currentYear = datetime.now().year
    currentYear

    dp_df = x.T
    dp_df['discount period'] = 0
    dp_df_old = dp_df.loc[dp_df.index<currentYear]
    dp_df_new = dp_df.loc[dp_df.index>=currentYear]
    for index, row in dp_df_new.iterrows():
        dp_df_new['discount period'].iloc[1:] = dp_df_new['discount period'].shift(1) +1
    dp_df_final = dp_df_old.append(dp_df_new)

    
    dp_df_final['Discount Factor'] = 1/(1+discount_rate)**dp_df_final['discount period']
    dp_df_final['PV of FCF'] = dp_df_final['FCF']*dp_df_final['Discount Factor'] 
    return dp_df_final

In [12]:
def intrinsic_value(discount_rate):
    dcf_valuation = pd.DataFrame(columns = ['sum PV of FCF', 'TV of FCF', 'PV of TV', 'Cash to Debt', 'Number of Shares', 'Discount Rate', 'Terminal Growth', 'Total Market Cap', 'Stock Price'])
    dcf_valuation.loc[1, 'sum PV of FCF'] = dp_df_final['PV of FCF'].sum()
    dcf_valuation.loc[1, 'Discount Rate'] = discount_rate
    dcf_valuation.loc[1, 'Number of Shares'] = is_df.T['Basic Average Shares'].iloc[-2]
    dcf_valuation.loc[1, 'Terminal Growth'] = 0.01
    dcf_valuation.loc[1, 'Cash to Debt'] = bs_df.T['Total Assets'].iloc[-1] - bs_df.T['Total Liabilities Net Minority Interest'].iloc[-1]
    dcf_valuation.loc[1, 'TV of FCF'] = int(dp_df_final['FCF'].sum()*(1+0)/(dcf_valuation['Discount Rate'] - dcf_valuation['Terminal Growth']))
    dcf_valuation.loc[1, 'PV of TV']= int(dcf_valuation['TV of FCF']*dp_df_final['Discount Factor'].iloc[-1])
    dcf_valuation.loc[1, 'Total Market Cap'] = int(dcf_valuation['sum PV of FCF']+dcf_valuation['PV of TV']+dcf_valuation['Cash to Debt'])
    dcf_valuation.loc[1, 'Stock Price'] = int(dcf_valuation['Total Market Cap']/dcf_valuation['Number of Shares'])
    return dcf_valuation

In [13]:
if __name__ == '__main__':
    fcf_df = fcf_calc(is_df)
    fcf_df2 = int_years(fcf_df)
    fcf_df_clean = fcf_fwd(fcf_df2)
    dcf_data = int_years(is_df)
    dcf_df = dcf_calc(dcf_data)
    discount_rate = .1
    dp_df_final = discount_period(dcf_df,discount_rate)
    dcf_valuation = intrinsic_value(discount_rate)

KeyError: 'Tax Provision'

In [ ]:
dcf_valuation.T

In [ ]:
'TESTING'
#current_year = 2020
#dcf_df.T.reset_index().loc[dcf_df.T.reset_index()['Breakdown'] >= current_year]

In [ ]:
############################################################
#Forget the search bar, just make the financial dashboard
############################################################

In [ ]:
app = dash.Dash(__name__)

#FCF for Dash
dash_is_df = fcf_df.T.reset_index()
available_is_indicators = fcf_df.index.unique()

#Balance Sheet for DASH
dash_bs_df = bs_df.T.reset_index()
dash_bs_df['Total Debt']= -dash_bs_df['Total Debt']
available_bs_indicators = bs_df.index.unique()

#DCF for DASH
dash_dcf_df = dcf_df.T.reset_index().rename(columns = {'Breakdown':'index'})

#analyst estimates
dash_rev_df = revenue_df.set_index('Revenue Estimate').T.reset_index().loc[2:]

app.layout = html.Div([
    html.Div([
    dcc.Graph(id='revenue', 
                  figure = px.line(dash_is_df, 
                                   x = 'index', 
                                   y = ['Total Revenue', 'Operating Income']),
              style={'width': '39%', 'height':'auto','display': 'inline-block'}),
    
        dcc.Graph(id='revenue-est', 
                  figure = px.line(dash_rev_df, 
                                   x = 'index',
                                   y = ['Low Estimate', 'Avg. Estimate', 'High Estimate']),
              style={'width': '39%', 'height':'auto','display': 'inline-block'})
    
    ]),
    
    html.Div([
        dcc.Graph(id='is_df', 
                  figure = px.bar(dash_is_df, 
                                   x = 'index', 
                                   y = ['Total Revenue', 'Cost of Revenue']),
              style={'width': '39%', 'height':'auto','display': 'inline-block'}),
    
    dcc.Graph(id = 'bs_df',
              figure = px.bar(dash_bs_df, 
                               x = 'index', 
                               y = ['Total Assets', 'Total Debt']),
              style={'width': '39%', 'height':'auto','display': 'inline-block'})\
    ]),
    
    html.Div([
    dcc.Graph(id='ebit-tax', 
                  figure = px.bar(dash_dcf_df, 
                                   x = 'index', 
                                   y = ['EBIT', 'Tax Provision']),
              style={'width': '39%', 'height':'auto','display': 'inline-block'}), 
     dcc.Graph(id='fcf-wc', 
                  figure = px.bar(dash_dcf_df, 
                                   x = 'index', 
                                   y = ['FCF', 'Working Capital']),
              style={'width': '39%', 'height':'auto','display': 'inline-block'})
        
    ])

    
    
    
    ])


app.run_server(debug=True, use_reloader=False)